In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

# headers = {
#     'User-Agent': 'Mozilla/5.0'
# }

# Daftar kategori dan tag URL untuk tiga sumber berita
kategori_list = {
    'Non-Sepak Bola': ['bulutangkis', 'basket', 'voli'],
    'Liga Inggris': ['liga-inggris'],
    'Liga Indonesia': ['liga-1'],
    'Liga Spanyol': ['la-liga'],
    'Liga Italia': ['serie-a'],
    'Liputan6': ['liputan6-sepakbola', 'liputan6-olahraga'],
    'Kompas': ['kompas-olahraga', 'kompas-bola']
}

# Daftar sumber berita untuk digunakan dalam scraping
sources = {
    'Detik': 'https://www.detik.com/tag/',
    'Liputan6': 'https://www.liputan6.com/tag/',
    'Kompas': 'https://www.kompas.com/tag/'
}

semua_berita = []
nomor = 1

# Fungsi untuk mengecek dan memproses artikel
def proses_artikel(link, kategori):
    try:
        r_artikel = requests.get(link, headers=headers)
        soup_artikel = BeautifulSoup(r_artikel.text, 'html.parser')

        # Ambil isi teks berita
        paragraphs = soup_artikel.select('div.detail__body-text p')  # Sesuaikan selector untuk setiap sumber
        isi_teks = ' '.join(p.get_text(strip=True) for p in paragraphs)

        # Ambil tanggal artikel
        if 'detik.com' in link:
            tanggal_artikel = soup_artikel.select_one('div.date')
            format_tanggal = "%d %b %Y"
        elif 'liputan6.com' in link:
            tanggal_artikel = soup_artikel.select_one('div.date_publish')
            format_tanggal = "%d %b %Y"
        elif 'kompas.com' in link:
            tanggal_artikel = soup_artikel.select_one('div.date')
            format_tanggal = "%d %b %Y"

        if tanggal_artikel:
            try:
                tanggal_artikel = datetime.strptime(tanggal_artikel.get_text().strip(), format_tanggal)
                if (datetime.now() - tanggal_artikel).days <= 180:  # Pastikan artikel dalam 6 bulan terakhir
                    if len(isi_teks.split()) >= 100:  # Pastikan teks lebih dari 100 kata
                        return {
                            'No': nomor,
                            'Text': isi_teks,
                            'URL': link,
                            'Kategori': kategori
                        }
            except ValueError:
                print(f"❗ Format tanggal tidak dikenali: {tanggal_artikel.get_text()}")
        return None
    except Exception as e:
        print(f"❗ Error mengambil {link}: {e}")
        return None

# Mengambil berita dari setiap kategori dan sumber
for kategori, tag_list in kategori_list.items():
    for tag in tag_list:
        print(f"\n🔄 Mengambil berita dari tag: {tag} (Kategori: {kategori})")

        # Iterasi untuk mengambil berita dari setiap sumber
        for sumber, base_url in sources.items():
            halaman = 1
            berita_per_kategori = 0
            while halaman <= 3 and berita_per_kategori < 25:  # Max 3 halaman, 25 berita per kategori
                url = f"{base_url}{tag}?page={halaman}"
                res = requests.get(url, headers=headers)
                soup = BeautifulSoup(res.text, 'html.parser')

                # Ambil semua link artikel
                links = soup.select("article a")
                for a in links:
                    link = a['href']
                    # Cek dan proses artikel
                    berita = proses_artikel(link, kategori)
                    if berita:
                        if link not in [b['URL'] for b in semua_berita]:
                            semua_berita.append(berita)
                            nomor += 1
                            berita_per_kategori += 1
                            if berita_per_kategori >= 25:
                                break
                halaman += 1
                time.sleep(2)

print(f"\n✅ Total berita terkumpul: {len(semua_berita)}")

# Simpan ke DataFrame dan ekspor ke CSV
df_final = pd.DataFrame(semua_berita)
df_final.to_csv("berita_olahraga_scraped.csv", index=False)
print("\n📁 Data berhasil disimpan ke 'berita_olahraga_scraped.csv'")



🔄 Mengambil berita dari tag: bulutangkis (Kategori: Non-Sepak Bola)

🔄 Mengambil berita dari tag: basket (Kategori: Non-Sepak Bola)

🔄 Mengambil berita dari tag: voli (Kategori: Non-Sepak Bola)

🔄 Mengambil berita dari tag: liga-inggris (Kategori: Liga Inggris)
